## Notebook to keep track of tabular data

### Raw tabular data is here
    `/home/nikhil/projects/Parkinsons/ppmi/tabular`

### Currently collating and tracking following assessments:
    - UPDRS
    - MoCA

### Notes
    - MDS-UPDRS_Part_I.csv has `NP1RTOT` i.e MDS-UPDRS Part I (Rater Completed) Total Score
    - MDS-UPDRS_Part_I_Patient_Questionnaire.csv has `NP1PTOT` i.e. MDS-UPDRS Part I (Patient Questionnaire) Total Score

In [1]:
import numpy as np
import pandas as pd
from itertools import product

In [2]:
tabular_dir = "/home/nikhil/projects/Parkinsons/ppmi/tabular"
ppmi_demographics_dir = f"{tabular_dir}/demographics"
ppmi_study_data_dir = f"{tabular_dir}/study_data"

# Available clinical visits (These are differently named than imaging sessions (i.e. session.csv))
# These are generated based on Vincent's notebooks/subject_profiling.ipynb
visits_csv = f"{ppmi_demographics_dir}/clinical_visits.csv" 

clinical_assessment_files = {
                "updrs_1": "MDS-UPDRS_Part_I.csv",
                "updrs_1_PQ": "MDS-UPDRS_Part_I_Patient_Questionnaire.csv",
                "updrs_2_PQ": "MDS_UPDRS_Part_II__Patient_Questionnaire.csv",
                "updrs_3": "MDS_UPDRS_Part_III.csv",
                "updrs_4_MC":"MDS-UPDRS_Part_IV__Motor_Complications.csv",
                "moca":"Montreal_Cognitive_Assessment__MoCA_.csv"
}

bagel_csv = f"{tabular_dir}/bagel.csv"

### PPMI mr_proc manifest

In [3]:
current_mr_proc_manifest_csv = f"{ppmi_demographics_dir}/mr_proc_manifest.csv"
current_mr_proc_manifest_df = pd.read_csv(current_mr_proc_manifest_csv)

n_participants = len(current_mr_proc_manifest_df["participant_id"].unique())
print(f"n_participants: {n_participants}")

dx_groups = current_mr_proc_manifest_df["group"].unique()
print(f"groups: {dx_groups}")
select_dex_groups = ['PD', 'Prodromal', 'Control', 'SWEDD']
print(f"select groups: {select_dex_groups}")
current_mr_proc_manifest_df = current_mr_proc_manifest_df[current_mr_proc_manifest_df["group"].isin(select_dex_groups)]

n_participants = len(current_mr_proc_manifest_df["participant_id"].unique())
print(f"n_participants: {n_participants}")
current_mr_proc_manifest_df.head()

n_participants: 1240
groups: ['Phantom' 'PD' 'Prodromal' 'Control' 'SWEDD' 'GenReg Unaff' 'AV133']
select groups: ['PD', 'Prodromal', 'Control', 'SWEDD']
n_participants: 1216


,participant_id,age,sex,group
3,sub-100001,67,M,PD
4,sub-100005,53,M,PD
5,sub-100006,56,F,PD
6,sub-100007,67,M,PD
7,sub-100012,66,F,PD


### Baseline counts
    - BIDS participants.tsv: n=560
    - mr_proc_manifest.csv: n=1240 (n=1216 unique)

In [4]:
mr_proc_ids = current_mr_proc_manifest_df["participant_id"].unique()
len(mr_proc_ids)

1216

### Available clinical visits (these are different than imaging sessions)
- Need to select a subset you expect data for 

In [4]:
visit_df = pd.read_csv(visits_csv)
visit_code_list = ["BL", "SC"] + list(visit_df[visit_df['visit_code'].str.contains('V')]["visit_code"].values)
print(f"Number of total possible visits: {len(visit_code_list)}")
visit_df.head()

Number of total possible visits: 20


,visit_code,visit_id
0,BL,0
1,V04,4
2,V06,6
3,V08,8
4,V10,10


### Read all UPDRS scores

In [5]:
updrs_part_1_df = pd.read_csv(f"{ppmi_study_data_dir}/{clinical_assessment_files['updrs_1']}")
updrs_part_1_df.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,NP1RTOT,ORIG_ENTRY,LAST_UPDATE
0,272451201,3000,BL,NUPDRS1,02/2011,1.0,1,0,1,1,0,0,3.0,02/2011,2020-06-25 16:02:19.0
1,338701901,3000,V04,NUPDRS1,03/2012,1.0,0,0,1,0,0,0,1.0,03/2012,2020-06-25 16:02:21.0
2,385008801,3000,V06,NUPDRS1,02/2013,1.0,1,0,1,1,0,0,3.0,02/2013,2020-06-25 16:02:22.0
3,437130601,3000,V08,NUPDRS1,03/2014,1.0,1,0,0,2,0,0,3.0,05/2014,2020-06-25 16:02:22.0
4,512466501,3000,V10,NUPDRS1,03/2015,1.0,0,0,0,0,0,0,0.0,03/2015,2020-06-25 16:02:23.0


In [6]:
updrs_part_1_PQ_df = pd.read_csv(f"{ppmi_study_data_dir}/{clinical_assessment_files['updrs_1_PQ']}")
updrs_part_1_PQ_df.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP1SLPN,NP1SLPD,NP1PAIN,NP1URIN,NP1CNST,NP1LTHD,NP1FATG,NP1PTOT,ORIG_ENTRY,LAST_UPDATE
0,272451501,3000,BL,NUPDRS1P,02/2011,1,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,02/2011,2020-06-25 16:02:19.0
1,338702301,3000,V04,NUPDRS1P,03/2012,1,0.0,2.0,0.0,1.0,0.0,0.0,1.0,4.0,03/2012,2020-06-25 16:02:21.0
2,385009201,3000,V06,NUPDRS1P,02/2013,1,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,02/2013,2020-06-25 16:02:22.0
3,437130801,3000,V08,NUPDRS1P,03/2014,1,3.0,2.0,0.0,1.0,0.0,0.0,0.0,6.0,05/2014,2020-06-25 16:02:22.0
4,512466901,3000,V10,NUPDRS1P,03/2015,1,1.0,1.0,1.0,1.0,1.0,0.0,1.0,6.0,03/2015,2020-06-25 16:02:23.0


In [7]:
updrs_part_2_PQ_df = pd.read_csv(f"{ppmi_study_data_dir}/{clinical_assessment_files['updrs_2_PQ']}")
updrs_part_2_PQ_df.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP2SPCH,NP2SALV,NP2SWAL,NP2EAT,...,NP2HWRT,NP2HOBB,NP2TURN,NP2TRMR,NP2RISE,NP2WALK,NP2FREZ,NP2PTOT,ORIG_ENTRY,LAST_UPDATE
0,272451701,3000,BL,NUPDRS2P,02/2011,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02/2011,2020-06-25 16:02:19.0
1,338702901,3000,V04,NUPDRS2P,03/2012,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,03/2012,2020-06-25 16:02:22.0
2,385009601,3000,V06,NUPDRS2P,02/2013,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,02/2013,2020-06-25 16:02:22.0
3,437131001,3000,V08,NUPDRS2P,03/2014,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,05/2014,2020-06-25 16:02:22.0
4,512467901,3000,V10,NUPDRS2P,03/2015,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,03/2015,2020-06-25 16:02:23.0


In [8]:
updrs_part_3_df = pd.read_csv(f"{ppmi_study_data_dir}/{clinical_assessment_files['updrs_3']}")
updrs_part_3_df.head()

/home/nikhil/anaconda3/envs/parkinsons/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,17,18,19,20,21,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,PDMEDDT,PDMEDTM,PDSTATE,EXAMTM,DBS_STATUS,...,DYSKIRAT,NHY,PDTRTMNT,DBSONTM,DBSOFFTM,HRPOSTMED,HRDBSOFF,HRDBSON,ORIG_ENTRY,LAST_UPDATE
0,272451901,3000,BL,NUPDRS3,02/2011,NaN,NaN,NaN,13:17:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,02/2011,2020-06-25 16:02:19.0
1,338703101,3000,V04,NUPDRS3,03/2012,NaN,NaN,NaN,13:47:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,03/2012,2020-06-25 16:02:22.0
2,385009801,3000,V06,NUPDRS3,02/2013,NaN,NaN,NaN,12:22:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,02/2013,2020-06-25 16:02:22.0
3,437131401,3000,V08,NUPDRS3,03/2014,NaN,NaN,NaN,13:22:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,05/2014,2020-06-25 16:02:22.0
4,512469901,3000,V10,NUPDRS3,03/2015,NaN,NaN,NaN,11:43:00,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,03/2015,2020-06-25 16:02:23.0


In [9]:
updrs_part_4_MC_df = pd.read_csv(f"{ppmi_study_data_dir}/{clinical_assessment_files['updrs_4_MC']}")
updrs_part_4_MC_df.head()


,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,NP4WDYSK,NP4WDYSKDEN,NP4WDYSKNUM,NP4WDYSKPCT,NP4DYSKI,...,NP4OFFPCT,NP4FLCTI,NP4FLCTX,NP4DYSTN,NP4DYSTNDEN,NP4DYSTNNUM,NP4DYSTNPCT,NP4TOT,ORIG_ENTRY,LAST_UPDATE
0,6a7cf647-eead-4890-8514-cb68e5d0cd47,3001,R17,NUPDRS4,11/2021,1,4.0,16.0,25.0,0,...,0.0,0,0,0,0.0,0.0,0.0,1.0,12/2021,2021-12-02 00:00:00.0
1,473773301,3001,V09,NUPDRS4,11/2014,0,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,0.0,11/2014,2020-06-25 16:04:34.0
2,516499401,3001,V10,NUPDRS4,04/2015,0,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,0.0,04/2015,2020-06-25 16:04:35.0
3,588306101,3001,V12,NUPDRS4,06/2016,0,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,0.0,09/2016,2020-06-25 16:04:35.0
4,681769601,3001,V14,NUPDRS4,03/2018,1,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,NaN,NaN,1.0,04/2018,2020-06-25 16:04:36.0


### Read MoCA scores

In [10]:
MoCA_df = pd.read_csv(f"{ppmi_study_data_dir}/{clinical_assessment_files['moca']}")
MoCA_df.head()

,REC_ID,PATNO,EVENT_ID,PAG_NAME,INFODT,MCAALTTM,MCACUBE,MCACLCKC,MCACLCKN,MCACLCKH,...,MCAREC5,MCADATE,MCAMONTH,MCAYR,MCADAY,MCAPLACE,MCACITY,MCATOT,ORIG_ENTRY,LAST_UPDATE
0,269556101,3000,SC,MOCA,01/2011,1,1,1,1,1,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,27.0,01/2011,2020-06-25 16:02:21.0
1,338704501,3000,V04,MOCA,03/2012,1,1,1,1,1,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,29.0,03/2012,2020-06-25 16:02:21.0
2,385012501,3000,V06,MOCA,02/2013,1,1,1,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,28.0,02/2013,2020-06-25 16:02:22.0
3,437134401,3000,V08,MOCA,03/2014,1,1,1,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,30.0,05/2014,2020-06-25 16:02:22.0
4,512480201,3000,V10,MOCA,03/2015,1,1,1,1,1,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,29.0,03/2015,2020-06-25 16:02:23.0


### Merge demographics, all UPDRS and MoCA total Scores into single CSV

In [11]:
participant_ids = current_mr_proc_manifest_df["participant_id"].values
a,b = zip(*list(product(participant_ids,visit_code_list)))

bagel_df = pd.DataFrame()
bagel_df["participant_id"] = a
bagel_df["EVENT_ID"] = b

# merge manifest df 
baseline_visit = "BL"
current_mr_proc_manifest_df["EVENT_ID"] = baseline_visit
bagel_df = pd.merge(bagel_df, current_mr_proc_manifest_df, on=["participant_id","EVENT_ID"], how="left")

# merge updrs and moca
total_col_dict = {
    "updrs_1": "NP1RTOT",
    "updrs_1_PQ": "NP1PTOT",
    "updrs_2_PQ": "NP2PTOT",
    "updrs_3":"NP3TOT",
    "updrs_4_MC":"NP4TOT",
    "moca": "MCATOT"
}

for k,v in total_col_dict.items():
    score_df = pd.read_csv(f"{ppmi_study_data_dir}/{clinical_assessment_files[k]}")
    score_df["participant_id"] = "sub-" + score_df["PATNO"].astype(str)
    bagel_df = pd.merge(bagel_df, score_df[["participant_id","EVENT_ID",v]], 
                        on=["participant_id","EVENT_ID"], how="left")

    # Assign availability status cols
    bagel_df[f"{v}_status"] = ~bagel_df[v].isna()

print(f"bagel df shape: {bagel_df.shape}")
bagel_df.head()

bagel df shape: (26204, 17)


/home/nikhil/anaconda3/envs/parkinsons/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,17,18,19,20,21,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,participant_id,EVENT_ID,age,sex,group,NP1RTOT,NP1RTOT_status,NP1PTOT,NP1PTOT_status,NP2PTOT,NP2PTOT_status,NP3TOT,NP3TOT_status,NP4TOT,NP4TOT_status,MCATOT,MCATOT_status
0,sub-100001,BL,67.0,M,PD,1.0,True,6.0,True,10.0,True,24.0,True,NaN,False,NaN,False
1,sub-100001,SC,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
2,sub-100001,V04,NaN,NaN,NaN,2.0,True,4.0,True,8.0,True,33.0,True,NaN,False,NaN,False
3,sub-100001,V06,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False
4,sub-100001,V08,NaN,NaN,NaN,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False,NaN,False


##

### Save bagel_df


In [12]:
save_bagel = False

print(f"Save path: {bagel_csv}")
if save_bagel:
    bagel_df.to_csv(bagel_csv)


Save path: /home/nikhil/projects/Parkinsons/ppmi/tabular/bagel.csv


In [17]:
bagel_df.groupby(["EVENT_ID"]).sum().reset_index()[["EVENT_ID","MCATOT_status"]]

,EVENT_ID,MCATOT_status
0,BL,31
1,SC,1163
2,V01,1
3,V02,0
4,V03,2
5,V04,1121
6,V05,1
7,V06,1050
8,V07,0
9,V08,944
